In [0]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler, StringIndexer, VectorIndexer, OneHotEncoder
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

spark = SparkSession.builder.appName('lg').getOrCreate()

In [0]:
customers = spark.read.csv('/FileStore/tables/customer_churn.csv', inferSchema=True, header=True)

In [0]:
customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [0]:
customers.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|      Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
|     

In [0]:
customers.columns

Out[5]: ['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [0]:
my_cols = customers.select(['Age', 'Total_Purchase', 'Years', 'Num_Sites', 'Company', 'Churn'])

In [0]:
my_final_data = my_cols.na.drop()

In [0]:
my_final_data.show()

+----+--------------+-----+---------+--------------------+-----+
| Age|Total_Purchase|Years|Num_Sites|             Company|Churn|
+----+--------------+-----+---------+--------------------+-----+
|42.0|       11066.8| 7.22|      8.0|          Harvey LLC|    1|
|41.0|      11916.22|  6.5|     11.0|          Wilson PLC|    1|
|38.0|      12884.75| 6.67|     12.0|Miller, Johnson a...|    1|
|42.0|       8010.76| 6.71|     10.0|           Smith Inc|    1|
|37.0|       9191.58| 5.56|      9.0|          Love-Jones|    1|
|48.0|      10356.02| 5.12|      8.0|        Kelly-Warren|    1|
|44.0|      11331.58| 5.23|     11.0|   Reynolds-Sheppard|    1|
|32.0|       9885.12| 6.92|      9.0|          Singh-Cole|    1|
|43.0|       14062.6| 5.46|     11.0|           Lopez PLC|    1|
|40.0|       8066.94| 7.11|     11.0|       Reed-Martinez|    1|
|30.0|      11575.37| 5.22|      8.0|Briggs, Lamb and ...|    1|
|45.0|       8771.02| 6.64|     11.0|    Figueroa-Maynard|    1|
|45.0|       8988.67| 4.8

In [0]:
assembler = VectorAssembler(inputCols=['Age', 'Total_Purchase', 'Years', 'Age', 'Num_Sites'], outputCol='features')

In [0]:
output = assembler.transform(customers)

In [0]:
final_data = output.select('features', 'churn')

In [0]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [0]:
log_reg_customer_churn = LogisticRegression(labelCol='churn')

In [0]:
fitted_churn_model = log_reg_customer_churn.fit(train_data)

In [0]:
training_summary = fitted_churn_model.summary

In [0]:
training_summary.predictions.describe().show()

+-------+-------------------+------------------+
|summary|              churn|        prediction|
+-------+-------------------+------------------+
|  count|                629|               629|
|   mean|0.17488076311605724|0.1287758346581876|
| stddev|0.38016740374235986|0.3352182419463142|
|    min|                0.0|               0.0|
|    max|                1.0|               1.0|
+-------+-------------------+------------------+



In [0]:
pred_and_labels = fitted_churn_model.evaluate(test_data)

In [0]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[26.0,8787.39,5.4...|    1|[1.24918389333961...|[0.77715855723495...|       0.0|
|[26.0,8939.61,4.5...|    0|[6.12668436836594...|[0.99782094895410...|       0.0|
|[29.0,8688.17,5.7...|    1|[3.02394757973150...|[0.95364434821902...|       0.0|
|[29.0,10203.18,5....|    0|[3.97133521560907...|[0.98150043458215...|       0.0|
|[29.0,13240.01,4....|    0|[6.73485220725204...|[0.99881266057484...|       0.0|
|[30.0,6744.87,5.1...|    0|[3.36470150676569...|[0.96658297052467...|       0.0|
|[30.0,7960.64,2.7...|    1|[3.63416162706967...|[0.97427327820403...|       0.0|
|[30.0,10960.52,5....|    0|[2.67672198693172...|[0.93563900772089...|       0.0|
|[30.0,13473.35,3....|    0|[2.70649364713395...|[0.93740873353258...|       0.0|
|[31.0,11297.57,

In [0]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction' ,labelCol='churn')

In [0]:
AUC = my_eval.evaluate(pred_and_labels.predictions)

In [0]:
AUC

Out[58]: 0.8348484848484848

In [0]:
new_customers = spark.read.csv('/FileStore/tables/new_customers.csv', inferSchema=True, header=True)

In [0]:
final_lr_model = log_reg_customer_churn.fit(final_data)

In [0]:
test_new_customers = assembler.transform(new_customers)

In [0]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [0]:
final_results = final_lr_model.transform(test_new_customers)

In [0]:
final_results.select('Company', 'prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

